In [19]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

All the years before 2011 was getting "HTTP Error 403: Forbidden", so i used selenium

In [20]:
def find_data(year_url, year, category):

    url = year_url+f'{year}'

    if year > 2010:
        html = requests.get(url).content

    else:
        driver_path = "../../chrome_driver/chromedriver.exe"
        driver = webdriver.Chrome(executable_path=driver_path)
        driver.get(url);
        html_content =  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "section-scroll")))
        html = html_content.get_attribute('outerHTML')
        driver.quit()


    soup = bs(html, 'html.parser')
    div_card = soup.find('div', class_="cards")# this div is to isolate all the events links

    events = soup.findAll("div", class_="col-md-5 col-12 not__cell__767__full")
    dates = soup.findAll("div", class_="col-md-2 col-4 not__cell__767")
    cities = soup.findAll("div", class_="col-md-3 col-4 not__cell__767")
    ids = div_card.findAll('a', href=True)

    text_events = []
    text_dates = []
    text_cities = []
    text_ids = []

    for event in events: text_events.append(event.text[1:-1])
    for date in dates: text_dates.append(date.text[2:-3])
    for city in cities: text_cities.append(city.text[3:-2])
    for id in ids: text_ids.append(id['href'][-3:])

    event_dict = {
        "Event":text_events,
        "Date":text_dates,
        "City":text_cities,
        "Event_id":text_ids,
        "Category":category
    }

    df = pd.DataFrame(event_dict)

    return df

In [21]:
new_category_url = "https://iwf.sport/results/results-by-events/?event_year="
old_category_url = "https://iwf.sport/results/results-by-events/results-by-events-old-bw/?event_year="

In [22]:
new_category_years = list(range(2018, 2023 + 1)) # The website url year gos from 2018 till 2023
old_category_years = list(range(1998, 2018 + 1)) # The website url year gos from 1998 till 2018

In [23]:
def make_df(url, years, category):
    dataframes = []
    erros = []
    for year in years:
        try:
            dataframes.append(find_data(url, year,category))
        except:
            erros.append(year)

    df = pd.concat(dataframes)

    return [df, erros]

In [24]:
new_category_results = make_df(new_category_url, new_category_years, "new category")
old_category_results = make_df(old_category_url, old_category_years, "old category")

In [25]:
new_category_erros = new_category_results[1]
old_category_erros = old_category_results[1]

print(new_category_erros, old_category_erros)

[] [2009]


In [26]:
new_category_df = new_category_results[0]
old_category_df = old_category_results[0]

In [27]:
main_df = pd.concat([new_category_df, old_category_df])

Fixing the date

In [28]:
def fix_date(date_column):
    date_list=[]

    for date in date_column:
        date_int = datetime.strptime(date, "%b %d, %Y")
        date_list.append(date_int)
        
    return date_list

In [29]:
main_df["Date"] = fix_date(main_df["Date"])

Fixing cities

In [30]:
main_df.head()

,Event,Date,City,Event_id,Category
0,5th International Qatar Cup,2018-12-19,"DOHA, QAT",444,new category
1,5th International Solidarity Championships,2018-12-08,"CAIRO, EGY",445,new category
2,II International Senior CSLP Cup,2018-12-07,"Guayaquil, ECU",442,new category
3,II International Junior CSLP Cup,2018-12-07,"Guayaquil, ECU",443,new category
4,2018 IWF World Championships,2018-11-01,"Ashgabat, TKM",441,new category


In [31]:
def fix_place(column, type):

    new_places=[]

    if type == "City":
            
        for city in column:
            new_places.append(city[:-5])
    
    if type == "Country":

        for country in column:
            new_places.append(country[-3:])

    return new_places

In [32]:
main_df["Country"] = fix_place(main_df["City"], "Country")
main_df["City"] = fix_place(main_df["City"], "City")

In [33]:
main_df.head()

,Event,Date,City,Event_id,Category,Country
0,5th International Qatar Cup,2018-12-19,DOHA,444,new category,QAT
1,5th International Solidarity Championships,2018-12-08,CAIRO,445,new category,EGY
2,II International Senior CSLP Cup,2018-12-07,Guayaquil,442,new category,ECU
3,II International Junior CSLP Cup,2018-12-07,Guayaquil,443,new category,ECU
4,2018 IWF World Championships,2018-11-01,Ashgabat,441,new category,TKM


Fixing id

In [34]:
def fix_id(column):
    new_ids=[]
    ids_erros=[]

    for id in column:

        try:
            if id[0] == 'd': new_ids.append(int(id[2:]))
            if id[0] == '=': new_ids.append(int(id[1:]))
            else: new_ids.append(int(id))
        except:
            ids_erros.append(id)

    return [new_ids, ids_erros]

In [35]:
main_df["Event_id"] = fix_id(main_df["Event_id"])[0]

In [36]:
main_df.to_excel("../../data/events_date.xlsx", index=False)